In [1]:
#드라이브 마운트 - 구글 드라이브의 데이터를 가져올 때 가져오는 라이브러리
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
#필요한 모듈 import
import pandas as pd
import numpy as np
import os

In [48]:
#data 확인
for dirname, _, filenames in os.walk('/content/drive/MyDrive/MoveLens/'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

/content/drive/MyDrive/MoveLens/genome_scores.csv
/content/drive/MyDrive/MoveLens/link.csv
/content/drive/MyDrive/MoveLens/movie.csv
/content/drive/MyDrive/MoveLens/rating.csv
/content/drive/MyDrive/MoveLens/genome_tags.csv
/content/drive/MyDrive/MoveLens/tag.csv


In [49]:
#data load
movie=pd.read_csv('/content/drive/MyDrive/MoveLens/movie.csv') 
rating=pd.read_csv('/content/drive/MyDrive/MoveLens/rating.csv')
link=pd.read_csv('/content/drive/MyDrive/MoveLens/link.csv')
tag=pd.read_csv('/content/drive/MyDrive/MoveLens/tag.csv') 
genome_tags=pd.read_csv('/content/drive/MyDrive/MoveLens/genome_tags.csv') 
genome_scores=pd.read_csv('/content/drive/MyDrive/MoveLens/genome_scores.csv')

1) User 24가 Item 2를 어떻게 평가할지 예측한 점수 수정(70%)

In [50]:
#preprocessing 진행
#1. 500개 이상 평가한 user만 filtering 진행
x=rating['userId'].value_counts()>500
print(len(x[x]))
thin_rating=rating[rating['userId'].isin(x[x].index)] 
movie_pivot=thin_rating.pivot_table(columns='userId',index='movieId',values='rating')

#2. 50개 이상 평가된 movie만 filtering 진행
y=thin_rating['movieId'].value_counts()>50
print(len(y[y]))
thin_rating=thin_rating[thin_rating['movieId'].isin(y[y].index)] 
movie_pivot=thin_rating.pivot_table(columns='userId',index='movieId',values='rating')

#3. 결측치 0으로 채우기
movie_pivot.fillna(0, inplace=True)

#4. transpose
user_pivot=movie_pivot.transpose()
user_pivot

7441
9313


movieId,1,2,3,4,5,6,7,8,9,10,...,116887,116977,117176,117590,118696,118700,118900,118997,119141,119145
userId,,,,,,,,,,,,,,,,,,,,,
11,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,4.0,0.0,0.0,0.0,2.0,4.0,3.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,4.0,3.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58,5.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,4.0,3.5,3.0,0.0,0.0,0.0,2.5,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138397,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138406,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138411,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [130]:
#1) Cosine distance의 Weighted average
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

#item 기준으로 user들이 평가한 점수를 기반으로 NearestNeighbor 모델 생성 후 fitting 진행
movie_sparse = csr_matrix(movie_pivot)
model = NearestNeighbors(n_neighbors=10, algorithm='brute', metric='cosine')
model.fit(movie_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [131]:
#movie 2의 user 평가 점수
movie2_rating = movie_pivot.iloc[1,:].values.reshape(1,-1)
movie2_rating

array([[0., 0., 3., ..., 0., 0., 4.]])

In [132]:
#movie2와 비슷하게 uesr들에게 평가 받은 movie top 10 찾기
distance, suggestions = model.kneighbors(movie2_rating)
distance, suggestions

(array([[0.        , 0.22626448, 0.23797185, 0.23983944, 0.24134662,
         0.24469272, 0.2489989 , 0.25214387, 0.2534928 , 0.25541199]]),
 array([[   1,  438,  658, 1273,  326,  334,  337, 2112, 1037,  532]]))

In [133]:
#similarity = 1 - distance, similarity를 weight로 사용
similarity = 1 - distance[0, 1:]
print(similarity)

[0.77373552 0.76202815 0.76016056 0.75865338 0.75530728 0.7510011
 0.74785613 0.7465072  0.74458801]


In [134]:
#top 10 중 movie2를 제외한 movie 선택
top_similarity = movie_pivot.iloc[suggestions[0, 1:]]
top_similarity

userId,11,24,54,58,91,104,116,134,156,208,...,138270,138301,138307,138325,138382,138397,138406,138411,138437,138474
movieId,,,,,,,,,,,,,,,,,,,,,
480,5.0,3.0,5.0,4.0,3.5,0.0,4.0,4.0,5.0,4.5,...,4.0,3.0,3.5,5.0,5.0,4.5,4.0,4.0,0.0,5.0
780,5.0,4.0,5.0,2.0,3.0,0.0,1.0,3.0,5.0,4.0,...,3.0,1.5,3.0,3.0,5.0,5.0,3.0,2.5,4.5,0.0
1580,5.0,5.0,4.0,4.0,3.5,3.0,2.5,4.0,5.0,1.5,...,3.0,2.0,3.0,4.5,5.0,5.0,3.5,4.0,4.5,5.0
356,5.0,5.0,5.0,4.5,4.0,0.0,4.0,3.0,5.0,1.0,...,4.0,3.5,0.0,4.5,5.0,4.5,4.0,4.0,0.0,5.0
364,4.0,0.0,0.0,0.0,4.5,0.0,4.0,0.0,4.0,3.5,...,4.0,3.0,0.0,5.0,3.0,4.0,4.0,3.5,0.0,5.0
367,4.0,4.0,4.0,5.0,2.5,0.0,1.0,3.0,4.0,3.5,...,3.0,3.0,1.5,0.0,1.0,0.0,3.5,3.5,0.0,5.0
2571,5.0,5.0,5.0,5.0,3.5,1.0,4.0,3.0,5.0,3.0,...,3.0,3.5,4.0,4.0,5.0,4.5,4.0,5.0,5.0,5.0
1270,5.0,4.0,5.0,0.0,4.0,3.0,3.5,3.0,4.0,4.0,...,4.0,3.5,3.5,5.0,4.0,4.5,4.0,4.5,4.0,0.0
586,4.0,0.0,3.0,1.0,3.0,0.0,1.0,0.0,4.0,3.5,...,2.0,3.0,0.0,3.5,3.0,0.0,4.0,0.0,0.0,4.0


In [135]:
#계산할 때 사용할 weight의 sum 계산
weight_sum = similarity.sum()
weight_sum

6.799837327524503

In [136]:
#cosine distance를 사용했을 때의 weighted average 구하기
weight = [0] * len(top_similarity.columns)

for i in range(len(top_similarity.columns)):
    sum = 0
    for j in range(len(top_similarity)):
        sum += similarity[j] * top_similarity[top_similarity.columns[i]].iloc[j]
    weight[i] = sum / weight_sum
cosine_distance_est = movie_pivot.copy()
cosine_distance_est.loc[2] = weight
cosine_distance_est.loc[2]

cosine_distance_est.head()
print("weighted average of cosine distance:", round(cosine_distance_est[24][2], 2))

weighted average of cosine distance: 3.34


In [141]:
#2)Pearson distance의 Weighted average
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation

model = NearestNeighbors(n_neighbors=10, algorithm='brute', metric='correlation')
model.fit(movie_pivot)

NearestNeighbors(algorithm='brute', metric='correlation', n_neighbors=10)

In [142]:
#movie2의 user 평가 점수
movie2_rating = movie_pivot.iloc[1,:].values.reshape(1,-1)
movie2_rating

array([[0., 0., 3., ..., 0., 0., 4.]])

In [143]:
#movie2와 비슷하게 uesr들에게 평가 받은 movie top 10 찾기
distance, suggestions = model.kneighbors(movie2_rating)
distance, suggestions

(array([[2.22044605e-16, 6.39855555e-01, 6.49124026e-01, 6.56934514e-01,
         6.57942996e-01, 6.58187688e-01, 6.62915806e-01, 6.64941985e-01,
         6.67291488e-01, 6.67998476e-01]]),
 array([[   1,  137,  334, 2871, 2146,  532,  337,  288,  630, 1653]]))

In [151]:
#top 10 user 확인 => cosine과 다르게 similarity를 구하기 위해 1에서 뺄 필요가 없어서 movie2 제거 안함
top_similarity = movie_pivot.iloc[suggestions[0, 0:]]
top_similarity

userId,11,24,54,58,91,104,116,134,156,208,...,138270,138301,138307,138325,138382,138397,138406,138411,138437,138474
movieId,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,3.0,0.0,3.5,0.0,2.0,0.0,5.0,0.0,...,0.0,2.5,2.5,3.0,4.0,0.0,3.0,0.0,0.0,4.0
158,4.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0
364,4.0,0.0,0.0,0.0,4.5,0.0,4.0,0.0,4.0,3.5,...,4.0,3.0,0.0,5.0,3.0,4.0,4.0,3.5,0.0,5.0
3489,0.0,0.0,2.0,1.0,3.5,0.0,2.5,0.0,3.0,0.0,...,3.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
2617,0.0,0.0,4.0,4.0,3.0,2.0,2.5,3.0,5.0,0.0,...,0.0,1.0,3.0,2.5,5.0,0.0,0.0,2.5,4.0,0.0
586,4.0,0.0,3.0,1.0,3.0,0.0,1.0,0.0,4.0,3.5,...,2.0,3.0,0.0,3.5,3.0,0.0,4.0,0.0,0.0,4.0
367,4.0,4.0,4.0,5.0,2.5,0.0,1.0,3.0,4.0,3.5,...,3.0,3.0,1.5,0.0,1.0,0.0,3.5,3.5,0.0,5.0
317,0.0,4.0,0.0,3.5,3.5,0.0,1.5,0.0,4.0,0.0,...,0.0,0.0,0.0,3.0,5.0,0.0,0.0,3.5,0.0,5.0
736,5.0,0.0,4.0,4.0,3.0,0.0,1.0,0.0,4.0,2.5,...,1.0,1.0,3.5,3.0,5.0,0.0,0.0,2.0,0.0,0.0


In [152]:
# Pearson correlation 구하기, wetight으로 사용
weight_corr = np.corrcoef(top_similarity)[0]
weight_corr

array([1.        , 0.36014444, 0.35087597, 0.34306549, 0.342057  ,
       0.34181231, 0.33708419, 0.33505802, 0.33270851, 0.33200152])

In [153]:
#계산할 때 사용할 weight의 sum 계산, movie2 제외
weight_sum = weight_corr[1:].sum()
weight_sum

3.07480746502116

In [170]:
#pearson correlation을 사용했을 때의 weighted average 구하기, movie2 제외
weighted_est = [0] * len(top_similarity[1:].columns)
for i in range(len(top_similarity[1:].columns)):
    sum = 0
    for j in range(len(top_similarity[1:])):
        sum += weight_corr[1:][j] * top_similarity[1:][top_similarity[1:].columns[i]].iloc[j]
    weighted_est[i] = sum / weight_sum
movie_pivot_est = movie_pivot.copy()
movie_pivot_est.loc[2] = weighted_est
movie_pivot_est
print('weighted average of pearson correlation:', round(movie_pivot_est[24][2], 2))

weighted average of pearson correlation: 0.87


In [158]:
#3)User-User Collaborative Filtering by Pearson correlation
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation

# user들이 평가한 점수를 기반으로 NearestNeighbor 모델 생성 후 fitting 진행
model=NearestNeighbors(n_neighbors=10, algorithm='brute', metric='correlation') 
model.fit(user_pivot)

NearestNeighbors(algorithm='brute', metric='correlation', n_neighbors=10)

In [159]:
#user24의 movie 평가 점수
user24_rating = user_pivot.iloc[1,:].values.reshape(1,-1)
user24_rating

array([[4., 0., 0., ..., 0., 0., 0.]])

In [161]:
#user24와 유사한 uesr top 10을 nearest neighbor로 찾음
distances,suggestions=model.kneighbors(user24_rating) 
distances,suggestions

(array([[0.        , 0.43120639, 0.48029303, 0.49779175, 0.50549205,
         0.50683534, 0.50691569, 0.51226352, 0.51236606, 0.51363966]]),
 array([[   1, 5219, 3008, 2439, 5054,   15, 5987, 2599, 4054, 4563]]))

In [163]:
#user24와 유사한 top 10 user의 평점 확인
similar_user = user_pivot.iloc[suggestions[0]]
similar_user

movieId,1,2,3,4,5,6,7,8,9,10,...,116887,116977,117176,117590,118696,118700,118900,118997,119141,119145
userId,,,,,,,,,,,,,,,,,,,,,
24,4.0,0.0,0.0,0.0,2.0,4.0,3.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96551,5.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54745,5.0,0.0,3.0,0.0,3.0,5.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43849,3.0,3.0,3.0,0.0,3.0,4.0,0.0,0.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93580,4.0,4.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298,4.0,3.0,3.0,0.0,3.0,5.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110670,4.0,0.0,0.0,0.0,2.5,4.0,0.0,0.0,0.0,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47195,3.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75079,4.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [165]:
#user24 기준 피어슨유사도 비교 => Pearson correaltion의 Weighted average 구하기 위해
user24_similarity = np.corrcoef(similar_user)[0]
user24_similarity

array([1.        , 0.56879361, 0.51970697, 0.50220825, 0.49450795,
       0.49316466, 0.49308431, 0.48773648, 0.48763394, 0.48636034])

In [167]:
#계산할 때 사용할 weight의 sum 계산, user24 제외
user24_similarity_sum = user24_similarity[1:].sum()
user24_similarity_sum

4.533196516173944

In [171]:
#pearson correlation을 사용했을 때의 weighted average 구하기, movie2 제외
weighted_est = [0] * len(similar_user[1:].columns)
for i in range(len(similar_user[1:].columns)):
    sum = 0
    for j in range(len(similar_user[1:])):
        sum += user24_similarity[1:][j] * similar_user[1:][similar_user[1:].columns[i]].iloc[j]
    weighted_est[i] = sum / user24_similarity_sum

user_pivot_est = user_pivot.copy()
user_pivot_est.loc[24] = weighted_est
user_pivot_est

print('weighted average of pearson correlation by user-user CF:', round(user_pivot_est[2][24], 2))

weighted average of pearson correlation by user-user CF: 1.42
